# Gemma를 허깅페이스로부터 불러와 추론

In [1]:
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 

In [2]:
import os,torch, wandb
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from transformers import TrainingArguments, set_seed
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
set_seed(42)
import random
import pandas as pd
import numpy as np
from tqdm import tqdm

/home/sw_innov01/anaconda3/envs/gemma/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
new_model = "Coldbrew9/Fine-tuning-gemma-singleQ"

## Model Inference

In [4]:
import os,torch

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 출력
print('Device:', device)  # 출력결과: cuda
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 1 (GPU #2 한개 사용하므로)
# print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 0 (수정 필요)

Device: cuda
Count of using GPUs: 1


In [5]:
## 본인의 허깅페이스 토큰을 넣어주세요
HUGGINGFACE_AUTH_TOKEN = '**'

In [6]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
    ) # 4bit quantization

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(new_model,token=HUGGINGFACE_AUTH_TOKEN, padding_side="left")

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
        new_model,
        token = HUGGINGFACE_AUTH_TOKEN,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
model.config.use_cache = True
model.eval()

Loading checkpoint shards: 100%|███████████████████████████████████| 4/4 [00:09<00:00,  2.37s/it]


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
         

In [8]:
import pandas as pd

# Read the test data from a CSV file
test = pd.read_csv('datasets/test.csv')

def generate300(test_question):
    """
    Generate 300 new tokens based on the given test question.

    Args:
        test_question (str): The input test question.

    Returns:
        str: The parsed answer generated by the model.
    """
    text = f"### Question: {test_question} \n ### Answer:"
    questions.append(test_question)
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=300)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    parsed_ans = decoded.split('### Answer: ')[1]
    return parsed_ans

def generate200(test_question):
    """
    Generate 200 new tokens based on the given test question.

    Args:
        test_question (str): The input test question.

    Returns:
        str: The parsed answer generated by the model.
    """
    text = f"### Question: {test_question} \n ### Answer:"
    questions.append(test_question)
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    parsed_ans = decoded.split('### Answer: ')[1]
    return parsed_ans

questions = []
preds = []

# Iterate over each test question
for i, test_question in enumerate(tqdm(test['질문'])): 
    count = -1
    if '그리고' in test_question:
        split_questions = test_question.split('그리고')
        count = 0
    elif '또한,' in test_question:
        split_questions = test_question.split('또한, ')
        count = 1
    else:
        split_questions = [test_question]
    temp_ans = ""
    
    # Generate answers for each split question
    for question in split_questions:
        if len(split_questions) == 2:
            result = generate200(question)
        else:
            result = generate300(question)
        temp_ans += result
        if len(split_questions)==2 and count ==0:
            temp_ans += " 그리고 "
            count = -1
        if len(split_questions)==2 and count ==1:
            temp_ans += " 또한, "
            count = -1

    questions.append(test_question)
    print(f"[ {i}번 ]", test_question)
    print(f"[ 답변 ]", temp_ans)
    print("\n")
    preds.append(temp_ans)
    
print("총 생성된 답변 개수 : ", len(preds))

  1%|▍                                                         | 1/130 [00:54<1:57:27, 54.63s/it]

[ 0번 ] 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
[ 답변 ] 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다. 광명단페인트는 반사율이 높아서 더 밝은 공간으로 만들어주며, 방청산화철페인트는 방청제가 첨가되어 방청 효과를 내는 페인트입니다. 알미늄페인트는 금속 또는 철강 표면에 사용되어 내화 및 방청 효과를 가지는 페인트입니다. 역청질페인트는 강철, 철판, 강관 등 또한, 원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다. 따라서 원목사이딩을 선택할 때는 이러한 단점을 감수하고 상황에 맞게 고려해야 합니다. 또한 추가적으로 설치 및 유지보수에 대한 전문가 비용이 필요할 수 있다는 점도 고려해야 합니다.### 수정사항 추가합니다. 추가적으로 원목사이딩은 특히 유지보수가 어렵고 습기에 민감하다는 점을 감안해야 합니다. 유지보수에 대한 노력과 비용을 기울이지




  2%|▉                                                         | 2/130 [01:35<1:39:39, 46.71s/it]

[ 1번 ] 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
[ 답변 ] 도배지에 녹은 자국이 발생하는 원인은 녹오염으로, 이는 높은 습도나 누수로 인해 발생할 수 있습니다. 이러한 경우, 해당 부분을 청소하고 습도를 관리하는 것이 중요합니다. 또한, 누수가 발생하는 경우에는 보수가 필요할 수 있으므로 전문가의 도움을 받는 것이 좋습니다. 부분적인 혹은 전체적인 재작업이 필요할 수 있으니 상황을 진단한 후에 적절한 조치를 취하는 것이 좋습니다. 누수로 인한 도배지의 녹오염이 발생한 경우, 해당 부분을 청소하고 습도를 관리하는 것이 중요합니다. 또한, 누수가 발생하는 경우에는 보수가 필요할 수 있으므로 전문가의 도움을 받는 것이 좋습니다. 부분적인 혹은 전체적인 재작업이 필요할 수 있으니 상황을 진단한 후에 적절한 조치를 취하는 것이 좋습니다. 부분적인 혹은 전체적인 재작업이 필요할 수 있으니 상황을 진단한 후에 적절한 조치를 취하는 것이 좋습니다. 부분적인 혹은 전체적인 재작업이 필요할 수 있으니 상황을 진단한 후




  2%|█▎                                                        | 3/130 [02:30<1:46:45, 50.44s/it]

...

[ 127번 ] 평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요?
[ 답변 ] 수성 벽체용 탄성 방수 도료는 치밀한 표면구성과 탄성 있는 도막을 형성하여 우수한 방수 기능을 제공하는데 더해 크랙을 방지하고 물세척이 가능합니다. 이 도료는 콘크리트 시멘트 구조물의 내외부 면에 다양하게 활용되며 내구성이 뛰어나 반영구적인 방수 솔루션으로 각광받고 있습니다. 또한 수성이기 때문에 환경과 안전에 더욱 친숙하면서도 편리하게 사용할 수 있는 장점이 있습니다. 이러한 특징으로 수성 벽체용 탄성 방수 도료는 건축 구조물의 방수 처리에 매우 효과적으로 활용될 수 있습니다. 그 밖에도 지하실이나 외벽, 욕실과 같은 다양한 구조물에서 사용되어 왔으며, 향후에도 건축물의 내부 및 외부 표면에 적용될 수 있는 다재다능한 방수 솔루션으로 활용될 것으로 기대됩니다. 수성 벽체용 탄성 방수 도료는 건축물의 수명을 연장하고, 수분으로 인한 파손을 방지하여 건물의 안전성을 높이는 데 도움이 됩니다. 따라




 99%|███████████████████████████████████████████████████████▌| 129/130 [1:42:22<00:47, 47.74s/it]

[ 128번 ] 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요."
[ 답변 ] 석고 수정이 발생하는 주된 원인은 건물의 오래됨에 따른 물리적 변화입니다. 건물이 오래되면 벽을 이루고 있는 석고의 물리적 특성이 변화하여 수정이 발생할 수 있습니다. 또한, 건물 주변의 흙이나 기반부의 변화, 습도의 변화 등도 석고 수정을 유발할 수 있습니다. 이러한 요인들을 고려하여 건물의 변화를 신중하게 관리하는 것이 중요합니다. 수정을 방지하기 위해서는 건물의 안전하고 건강한 환경을 유지하는 것이 필요합니다. 수정이 발생했다면 전문가의 도움을 받아 적절한 조치를 취하는 것이 좋습니다. Prevention is better than cure! 수정을 방지하기 위해서는 건물의 안전하고 건강한 환경을 유지 그리고  이를 해결하기 위해서는 몇 가지 요령이 있습니다. 먼저, 양쪽 모서리에 위치한 책상을 이동하여 중앙에 위치시키는 것이 좋습니다. 이렇게 하면 공간 활용이 더 효율적이 될 수 있습니다. 또한, 책상 아래에 수납 공간을 만들어서 물건을 정리하고 깔끔한 모습을 유지하는 것이 좋습니다. 마지막으로, 벽면을 활용하여 수납장을 설치하여 물건을 정리하고 깔끔한 모습을 유지할 수 있습니다. 이러한 방법들을 활용하여 공간을 효율적으로 활용할 수 있습니다.  만약, 책상을 중앙에 위치시키기 어렵다면, 오른쪽 모서리에 위치한 책상을 이동하여 빈 공간을 만들어서 물건을 정




100%|████████████████████████████████████████████████████████| 130/130 [1:43:17<00:00, 47.67s/it]

[ 129번 ] 카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요?
[ 답변 ] 카페트의 기대수명은 6년입니다. 이는 일반적인 사용 상황에 따른 수명이며, 사용 빈도, 관리 정도, 실내 환경 등에 따라 달라질 수 있습니다. 따라서 카페트를 사용하는 동안에는 정기적인 청소와 유지보수를 통해 수명을 연장시킬 수 있습니다. 또한, 사용 환경의 습도와 온도 등도 카페트의 수명에 영향을 미칠 수 있으니 주의가 필요합니다.정기적인 관리와 유지보수를 통해 카페트의 수명을 연장시킬 수 있습니다. 또한, 사용 환경의 습도와 온도 등도 카페트의 수명에 영향을 미칠 수 있으니 주의가 필요합니다.정기적인 관리와 유지보수를 통해 카페트의 수명을 연 그리고 오리지널징크의 장점은 다양한 패턴과 디자인을 적용할 수 있다는 것입니다. 또한 친환경적이고 높은 내식성이 있어 수명이 다른 징크에 비해 길며 유지보수가 쉽다는 점이 있습니다. 뿐만 아니라 오리지널징크는 가벼우면서도 내부식성이 뛰어나서 다양한 용도에 적용할 수 있습니다. 이러한 이유로 오리지널징크는 건물 외벽 및 지붕재로 널리 사용되고 있습니다. 또한 오리지널징크는 금속의 유연성과 강도를 갖추고 있어 formability가 뛰어나며, 이러한 특성으로 인해 다양한 디자인에 적용하기에 용이합니다. 따라서 오리지널징크는 외관적으로 미적이며


총 생성된 답변 개수 :  130


In [13]:
len(questions)

326

In [14]:
df = pd.DataFrame({'question': test['질문'].to_list(), 'answer': preds})
df.to_csv(f'{new_model}Split_v2_qna.csv')

In [15]:
for i, test_question in enumerate(tqdm(test['질문'])):
    questions.append(test_question)
    print(f"[ {i}번 ]", test_question)
    print(f"[ 답변 ]", preds[i])
    print("\n")

100%|███████████████████████████████████████████████████████| 130/130 [00:00<00:00, 29553.36it/s]

[ 0번 ] 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
[ 답변 ] 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다. 광명단페인트는 반사율이 높아서 더 밝은 공간으로 만들어주며, 방청산화철페인트는 방청제가 첨가되어 방청 효과를 내는 페인트입니다. 알미늄페인트는 금속 또는 철강 표면에 사용되어 내화 및 방청 효과를 가지는 페인트입니다. 역청질페인트는 강철, 철판, 강관 등 또한, 원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다. 따라서 원목사이딩을 선택할 때는 이러한 단점을 감수하고 상황에 맞게 고려해야 합니다. 또한 추가적으로 설치 및 유지보수에 대한 전문가 비용이 필요할 수 있다는 점도 고려해야 합니다.### 수정사항 추가합니다. 추가적으로 원목사이딩은 특히 유지보수가 어렵고 습기에 민감하다는 점을 감안해야 합니다. 유지보수에 대한 노력과 비용을 기울이지


[ 1번 ] 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
[ 답변 ] 도배지에 녹은 자국이 발생하는 원인은 녹오염으로, 이는 높은 습도나 누수로 인해 발생할 수 있습니다. 이러한 경우, 해당 부분을 청소하고 습도를 관리하는 것이 중요합니다. 또한, 누수가 발생하는 경우에는 보수가 필요할 수 있으므로 전문가의 도움을 받는 것이 좋습니다. 부분적인 혹은 전체적인 재작업이 필요할 수 있으니 상황을 진단한 후에 적절한 조치를 취하는 것이 좋습니다. 누수로 인한 도배지의 녹오염이 발생한 경우, 해당 부분을 청소하고 습도를 관리하는 것이 중요합니다. 또한, 누수가 발

## Submission

In [16]:
len(preds)

130

In [17]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [18]:
submit = pd.read_csv('./sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

/tmp/ipykernel_3495677/1828452193.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.00485724  0.01085756  0.01782427  0.00940065 -0.00308758  0.01973073
 -0.01280303  0.00669193 -0.02320927  0.01074731  0.02489333 -0.01071081
 -0.02802906  0.0048238   0.03252767 -0.01784922  0.01414747 -0.05846553
 -0.03560882 -0.00356169  0.04677778  0.01725388 -0.0330446   0.03902486
  0.0166583  -0.00973707  0.00863511 -0.02555329  0.02217829 -0.01077144
  0.02739385 -0.00543527  0.00780968  0.06761812 -0.00278158 -0.0269175
 -0.00502757  0.01721529  0.05501264  0.00976747 -0.01976709 -0.00193022
  0.04314896  0.03519874  0.00263577 -0.05081276 -0.00631279 -0.08698884
 -0.0027458   0.02198513 -0.01298072  0.04118897  0.00585941 -0.00721182
  0.05404537  0.00979144  0.03823508  0.03680642  0.00034177  0.01823476
 -0.02078891  0.00579312  0.022941    0.03112095 -0.00383251 -0.00772633
  0.01794941  0.01887673  0.00909117 -0.

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.004857,0.041591,0.010666,0.006392,0.053562,0.030527,-0.046489,0.015795,0.017009,...,-0.011199,-0.041688,-0.033805,-0.041464,0.006364,0.000552,0.032671,0.009508,0.040476,0.031796
1,TEST_001,0.010858,0.012621,-0.032944,0.022089,0.104961,0.021058,0.016599,-0.034306,-0.000886,...,-0.023159,0.002144,0.028291,-0.026054,-0.022122,0.014385,-0.022416,-0.020754,0.043673,0.007664
2,TEST_002,0.017824,-0.025845,-0.037522,0.005318,0.145898,-0.006604,0.001171,-0.017473,0.038961,...,-0.013893,-0.022668,0.050859,-0.026597,-0.047666,0.001454,-0.033747,-0.028951,-0.042368,0.080022
3,TEST_003,0.009401,-0.006114,-0.004442,0.035187,0.048125,-0.018735,-0.078046,-0.020889,-0.006314,...,-0.047457,0.002225,0.027574,-0.051914,0.007330,0.006081,-0.041775,-0.048374,-0.009331,0.046276
4,TEST_004,-0.003088,-0.019108,-0.009798,-0.010762,0.111735,-0.021662,0.057287,0.044304,-0.017061,...,0.017085,0.003293,0.035827,0.011383,-0.044057,0.023593,0.006391,0.029857,0.005778,0.044461


In [19]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv(f'./{new_model}Split_v2.csv', index=False)